# 04 – Out-of-Sample Backtesting

Evaluate whether in-sample-optimised portfolios outperform simple benchmarks  
on **out-of-sample** (2024) data.

Portfolios compared:
1. **GMVP** – Global Minimum Variance (optimised in-sample)
2. **MSR**  – Maximum Sharpe Ratio   (optimised in-sample)
3. **Equal Weight** – 1/N benchmark
4. **SPY**  – S&P 500 ETF (market benchmark)

Metrics: annualised return, volatility, Sharpe ratio, max drawdown, Calmar ratio.

In [ ]:
import sys; sys.path.insert(0, '..')
import numpy as np
import pandas as pd

from src.data_handler  import load_data
from src.optimizer     import min_variance, max_sharpe
from src.metrics       import portfolio_daily_returns, cumulative_wealth, compare_portfolios
from src.visualization import plot_backtest, plot_drawdown

TICKERS   = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'JPM', 'SPY']
RF        = 0.04

data      = load_data(TICKERS, '2015-01-01', '2024-12-31', train_end='2023-12-31')
mu, cov   = data['mu'], data['cov']
test_ret  = data['test_returns']
n         = len(TICKERS)

print('In-sample period:  ', data['train_returns'].index[0].date(), '→', data['train_returns'].index[-1].date())
print('Out-of-sample:     ', test_ret.index[0].date(), '→', test_ret.index[-1].date())
print('OOS trading days:  ', len(test_ret))

In [ ]:
# ── Optimise weights IN-SAMPLE ────────────────────────────────────────────────
gmvp = min_variance(mu, cov)
msr  = max_sharpe(mu, cov, risk_free_rate=RF)

print('GMVP weights:')
print(gmvp['weights'].to_frame('weight').T.to_string(), '\n')
print('MSR weights:')
print(msr['weights'].to_frame('weight').T.to_string())

In [ ]:
# ── Apply weights OUT-OF-SAMPLE (fixed, no rebalancing) ───────────────────────
portfolios_dr = {}

portfolios_dr['GMVP']         = pd.Series(
    portfolio_daily_returns(gmvp['weights'].values, test_ret), index=test_ret.index)

portfolios_dr['Max Sharpe']   = pd.Series(
    portfolio_daily_returns(msr['weights'].values, test_ret),  index=test_ret.index)

portfolios_dr['Equal Weight'] = pd.Series(
    portfolio_daily_returns(np.ones(n)/n, test_ret),           index=test_ret.index)

# SPY-only portfolio (already one of our tickers)
spy_idx = TICKERS.index('SPY') if 'SPY' in TICKERS else None
if spy_idx is not None:
    w_spy = np.zeros(n); w_spy[spy_idx] = 1.0
    portfolios_dr['SPY only'] = pd.Series(
        portfolio_daily_returns(w_spy, test_ret), index=test_ret.index)

cum_returns = {k: cumulative_wealth(v) for k, v in portfolios_dr.items()}
for k, v in cum_returns.items():
    print(f'{k:16s}  final value: {v.iloc[-1]:.3f}')

In [ ]:
plot_backtest(cum_returns, title='Out-of-Sample Cumulative Wealth (2024)').show()

In [ ]:
plot_drawdown(portfolios_dr, title='Out-of-Sample Drawdown').show()

In [ ]:
# ── Performance metrics table ─────────────────────────────────────────────────
df_metrics, fmt = compare_portfolios(portfolios_dr, risk_free_rate=RF)
styled = df_metrics.T  # metrics as rows looks cleaner in notebooks
styled.style.format({
    'Annualised Return': '{:.2%}',
    'Annualised Vol':    '{:.2%}',
    'Sharpe Ratio':      '{:.3f}',
    'Max Drawdown':      '{:.2%}',
    'Calmar Ratio':      '{:.3f}',
})

In [ ]:
# ── Monthly rebalancing simulation ────────────────────────────────────────────
# Re-optimise each month using expanding in-sample window
from src.data_handler import compute_returns, annualise_stats

full_prices    = data['prices']
full_log_ret   = data['log_returns']
test_dates     = test_ret.index
monthly_bounds = pd.date_range(test_dates[0], test_dates[-1], freq='MS')

rebal_returns = []

for i, month_start in enumerate(monthly_bounds):
    # In-sample: everything before this month
    insample = full_log_ret.loc[:month_start].iloc[:-1]
    if len(insample) < 60:   # need minimum history
        continue
    mu_r, cov_r = annualise_stats(insample)
    msr_r = max_sharpe(mu_r, cov_r, risk_free_rate=RF)
    if msr_r['weights'] is None:
        continue

    # Out-of-sample slice for this month
    if i + 1 < len(monthly_bounds):
        month_end = monthly_bounds[i + 1]
    else:
        month_end = test_dates[-1]
    month_ret = test_ret.loc[month_start:month_end]
    dr = portfolio_daily_returns(msr_r['weights'].values, month_ret)
    rebal_returns.append(pd.Series(dr, index=month_ret.index))

if rebal_returns:
    rebal_series = pd.concat(rebal_returns)
    rebal_series = rebal_series[~rebal_series.index.duplicated()]
    portfolios_dr['MSR (monthly rebal)'] = rebal_series

cum_returns2 = {k: cumulative_wealth(v) for k, v in portfolios_dr.items()}
plot_backtest(cum_returns2, title='With Monthly Rebalancing vs Static').show()